In [3]:
!git clone https://github.com/angel-one/smartapi-python.git

Cloning into 'smartapi-python'...


In [4]:
!pip install -r smartapi-python/requirements_dev.txt

  Using cached attrs-23.1.0-py3-none-any.whl.metadata (11 kB)
  Using cached autobahn-23.6.2-py2.py3-none-any.whl
  Using cached Automat-22.10.0-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl.metadata (2.2 kB)
  Using cached cffi-1.15.1-cp311-cp311-win_amd64.whl.metadata (1.1 kB)
  Using cached charset_normalizer-3.1.0-cp311-cp311-win_amd64.whl.metadata (31 kB)
  Using cached constantly-15.1.0-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached cryptography-43.0.1-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
  Using cached hyperlink-21.0.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached idna-3.4-py3-none-any.whl.metadata (9.8 kB)
  Using cached incremental-22.10.0-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl.metadata (9.6 kB)
  Using cached logzero-1.7.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl.metadata (1.1 kB)
  Using cached pyotp-2.8.0-py3-none-an

  You can safely remove it manually.


In [5]:
!pip install smartapi-python

In [6]:
!pip install logzero

In [7]:
from SmartApi import SmartConnect
import pyotp
from logzero import logger

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [8]:
from dotenv import load_dotenv
import os
import json
import datetime as dt
from SmartApi import SmartConnect
import pytz
import time

In [9]:
load_dotenv()

True

In [14]:
API_KEY = os.getenv("API_KEY")
CLIENT_ID = os.getenv("CLIENT_ID")
PASSWORD = os.getenv("PASSWORD")
TOTP = os.getenv("TOTP")

In [15]:
try:
    # Create SmartConnect object
    obj = SmartConnect(api_key=API_KEY)

    # Generate TOTP
    totp = pyotp.TOTP(TOTP_SECRET).now()
    print("🔑 TOTP Code:", totp)

    # Perform login
    data = obj.generateSession(CLIENT_ID, PASSWORD, totp)

    print("✅ Login Successful!")
    print("Profile Data:", data)

    # Save tokens for later use
    jwt_token = data["data"]["jwtToken"]
    refresh_token = data["data"]["refreshToken"]

except Exception as e:
    print("❌ Login Failed:", str(e))

[I 250530 20:47:32 smartConnect:124] in pool


❌ Login Failed: 'NoneType' object has no attribute 'TOTP'


In [19]:
import json
import datetime as dt
import pytz
import time
import csv
import os
from dotenv import load_dotenv

In [49]:
INDEX = "NIFTY"
EXCHANGE = "NFO"
LOTSIZE = 75
IST = pytz.timezone("Asia/Kolkata")
MARKET_OPEN = dt.time(9, 15)
MARKET_CLOSE = dt.time(15, 30)

In [20]:
def fetch_instruments_from_file(filename="instruments.json"):
    with open(filename, "r") as f:
        return json.load(f)

In [36]:
def check_market_status():
    now = dt.datetime.now(IST)
    today = now.date()
    current_time = now.time()
    weekday = now.weekday()
    if weekday >= 5:
        return "closed", "Market closed (Weekend)"
    if current_time < MARKET_OPEN:
        return "preopen", "Market not opened yet"
    if current_time > MARKET_CLOSE:
        return "closed", "Market closed for the day"
    return "open", "Market is open"

In [37]:
def login():
    obj = SmartConnect(api_key=os.getenv("API_KEY"))
    totp_secret = os.getenv("TOTP_SECRET")
    totp = pyotp.TOTP(totp_secret).now()
    jwt = obj.generateSession(os.getenv("CLIENT_ID"), os.getenv("PASSWORD"), totp)
    print(f"✅ Logged in. JWT token obtained.")
    return obj

In [38]:
def get_nearest_expiry(instruments):
    today = dt.date.today()
    expiries = sorted({
        ins["expiry"] for ins in instruments
        if ins.get("name") == INDEX and ins.get("exchange") == EXCHANGE
    })
    for exp in expiries:
        exp_date = dt.datetime.strptime(exp, "%d-%b-%Y").date()
        if exp_date >= today:
            return exp
    return None

In [39]:
def get_atm_strike(obj):
    ltp_data = obj.ltpData(exchange="NSE", tradingsymbol=INDEX, symboltoken="99926000")
    spot_price = float(ltp_data["data"]["ltp"])
    return round(spot_price / 50) * 50

In [40]:
def get_option_symbol(instruments, strike, option_type, expiry):
    for ins in instruments:
        if (
            ins.get("strike") == strike and
            ins.get("symbol") == INDEX and
            ins.get("expiry") == expiry and
            ins.get("optiontype") == option_type
        ):
            return ins.get("tradingsymbol"), ins.get("symboltoken")
    return None, None

In [41]:
def place_order(obj, symbol, token, transaction_type, quantity):
    try:
        order_id = obj.placeOrder(
            variety="NORMAL",
            tradingsymbol=symbol,
            symboltoken=token,
            transactiontype=transaction_type,
            exchange=EXCHANGE,
            ordertype="MARKET",
            producttype="INTRADAY",
            duration="DAY",
            price="0",
            squareoff="0",
            stoploss="0",
            quantity=quantity
        )
        print(f"✅ Order Placed: {transaction_type} {symbol} | Order ID: {order_id}")
        return order_id
    except Exception as e:
        print(f"❌ Failed to place order: {e}")
        return None

In [43]:
def log_trade(action, symbol, price, quantity, trade_type, timestamp=None):
    if timestamp is None:
        timestamp = dt.datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S")
    df = pd.DataFrame([{
        "Timestamp": timestamp,
        "Action": action,
        "Symbol": symbol,
        "Price": price,
        "Quantity": quantity,
        "TradeType": trade_type
    }])
    if not os.path.isfile(CSV_LOG_FILE):
        df.to_csv(CSV_LOG_FILE, index=False)
    else:
        df.to_csv(CSV_LOG_FILE, mode='a', header=False, index=False)

In [45]:
def monitor_positions(obj, ce_symbol, ce_token, pe_symbol, pe_token, ce_entry_price, pe_entry_price, stoploss_pct, target_pct, trailing_pct, qty, trade_type):
    sl_ce = ce_entry_price * (1 - stoploss_pct / 100)
    sl_pe = pe_entry_price * (1 - stoploss_pct / 100)
    target_ce = ce_entry_price * (1 + target_pct / 100)
    target_pe = pe_entry_price * (1 + target_pct / 100)
    trail_ce = ce_entry_price
    trail_pe = pe_entry_price

    print(f"🔒 Initial SL CE: {sl_ce:.2f}, SL PE: {sl_pe:.2f}")
    print(f"🎯 Target CE: {target_ce:.2f}, Target PE: {target_pe:.2f}")
    print(f"🛡️ Trailing SL %: {trailing_pct}%")

    while True:
        try:
            ce_ltp = float(obj.ltpData(exchange=EXCHANGE, tradingsymbol=ce_symbol, symboltoken=ce_token)["data"]["ltp"])
            pe_ltp = float(obj.ltpData(exchange=EXCHANGE, tradingsymbol=pe_symbol, symboltoken=pe_token)["data"]["ltp"])
        except Exception as e:
            print(f"Error fetching LTP: {e}")
            time.sleep(5)
            continue

        if ce_ltp > trail_ce * (1 + trailing_pct / 100):
            trail_ce = ce_ltp
            sl_ce = trail_ce * (1 - stoploss_pct / 100)
            print(f"📈 CE new trail price: {trail_ce:.2f}, new SL: {sl_ce:.2f}")

        if pe_ltp > trail_pe * (1 + trailing_pct / 100):
            trail_pe = pe_ltp
            sl_pe = trail_pe * (1 - stoploss_pct / 100)
            print(f"📈 PE new trail price: {trail_pe:.2f}, new SL: {sl_pe:.2f}")
        if ce_ltp <= sl_ce:
            print(f"📉 CE hit Stoploss at {ce_ltp:.2f}")
            log_trade("EXIT", ce_symbol, ce_ltp, qty, trade_type)
            break
        if ce_ltp >= target_ce:
            print(f"🚀 CE hit Target at {ce_ltp:.2f}")
            log_trade("EXIT", ce_symbol, ce_ltp, qty, trade_type)
            break
        if pe_ltp <= sl_pe:
            print(f"📉 PE hit Stoploss at {pe_ltp:.2f}")
            log_trade("EXIT", pe_symbol, pe_ltp, qty, trade_type)
            break
        if pe_ltp >= target_pe:
            print(f"🚀 PE hit Target at {pe_ltp:.2f}")
            log_trade("EXIT", pe_symbol, pe_ltp, qty, trade_type)
            break

        time.sleep(5)

In [50]:
def run_strategy(lots=1, stoploss_pct=12.5, target_pct=35, trailing_pct=5, paper_trade=False):
    status_code, status_msg = check_market_status()
    print(f"🕒 Market Status: {status_msg}")
    if status_code != "open":
        print("⏳ Exiting - market not open.")
        return

    obj = login()

    instruments = fetch_instruments_from_file("instruments.json")
    print(f"📄 Loaded {len(instruments)} instruments.")

    expiry = get_nearest_expiry(instruments)
    if not expiry:
        print("❌ No valid expiry found!")
        return
    print(f"📆 Nearest expiry: {expiry}")

    atm_strike = get_atm_strike(obj)
    print(f"📈 ATM Strike: {atm_strike}")

    ce_symbol, ce_token = get_option_symbol(instruments, atm_strike, "CE", expiry)
    pe_symbol, pe_token = get_option_symbol(instruments, atm_strike, "PE", expiry)

    if not ce_symbol or not pe_symbol:
        print("❌ Could not find CE or PE option symbols.")
        return

    qty = lots * LOTSIZE

    if paper_trade:
        print(f"📝 Paper Trade: Simulating BUY {qty} of {ce_symbol} and {pe_symbol}")
        ce_ltp = float(obj.ltpData(exchange=EXCHANGE, tradingsymbol=ce_symbol, symboltoken=ce_token)["data"]["ltp"])
        pe_ltp = float(obj.ltpData(exchange=EXCHANGE, tradingsymbol=pe_symbol, symboltoken=pe_token)["data"]["ltp"])

        log_trade("BUY", ce_symbol, ce_ltp, qty, "Paper")
        log_trade("BUY", pe_symbol, pe_ltp, qty, "Paper")

        monitor_positions(obj, ce_symbol, ce_token, pe_symbol, pe_token,
                          ce_ltp, pe_ltp, stoploss_pct, target_pct, trailing_pct,
                          qty, "Paper")
    else:
        # Live trade orders
        ce_order_id = place_order(obj, ce_symbol, ce_token, "BUY", qty)
        pe_order_id = place_order(obj, pe_symbol, pe_token, "BUY", qty)
        if ce_order_id is None or pe_order_id is None:
            print("❌ Failed to place live orders, exiting.")
            return

        # Fetch LTP after placing orders for logging and monitoring
        ce_ltp = float(obj.ltpData(exchange=EXCHANGE, tradingsymbol=ce_symbol, symboltoken=ce_token)["data"]["ltp"])
        pe_ltp = float(obj.ltpData(exchange=EXCHANGE, tradingsymbol=pe_symbol, symboltoken=pe_token)["data"]["ltp"])

        log_trade("BUY", ce_symbol, ce_ltp, qty, "Live")
        log_trade("BUY", pe_symbol, pe_ltp, qty, "Live")

        monitor_positions(obj, ce_symbol, ce_token, pe_symbol, pe_token,
                          ce_ltp, pe_ltp, stoploss_pct, target_pct, trailing_pct,
                          qty, "Live")

if __name__ == "__main__":
    # Example usage:
    run_strategy(lots=1, paper_trade=True) 

🕒 Market Status: Market closed for the day
⏳ Exiting - market not open.


In [52]:
run_strategy(lots=1, paper_trade=True)

🕒 Market Status: Market closed for the day
⏳ Exiting - market not open.
